In [1]:
import pandas as pd
import os
# from transformers import BertTokenizer, BertModel
from bert_score import BERTScorer
import re
import pickle
import nltk
from nltk.translate import meteor
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import math

/Users/isabel/anaconda3/envs/nlp_metrics/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /Users/isabel/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/isabel/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/isabel/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /Users/isabel/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/isabel/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nlt

True

In [3]:
gpt3_load = './generatedDataFromGoogleDrive_gpt3/data/'
gpt4_load = './generatedDataFromGoogleDrive_gpt4/data/'
gpt3_save_bert = './metrics/gpt3_F1_BERT_scores.pkl'
gpt4_save_bert = './metrics/gpt4_F1_BERT_scores.pkl'
gpt3_save_meteor = './metrics/gpt3_meteor_scores.pkl'
gpt4_save_meteor = './metrics/gpt4_meteor_scores.pkl'
gpt3_save_sentiment = './metrics/gpt3_sentiment_scores.pkl'
gpt4_save_sentiment = './metrics/gpt4_sentiment_scores.pkl'

# Contextual Similarity - BERTScore

In [4]:
def bertScore(loadingFile, savingFile):
    # load all files from data folder (generated in Google Colab)
    met_dfs = {}
    unmet_dfs = {}
    for file in os.listdir(loadingFile):
        if '.csv' in file:
            
            if 'unmet' in file:
                unmet_dfs[file.replace('.csv', '')] = pd.read_csv(f'{loadingFile}{file}')
            else:
                met_dfs[file.replace('.csv', '')] = pd.read_csv(f'{loadingFile}{file}')
    nurse_notes = pd.read_excel('./fake_notes.xlsx')

    score_dictionary = {}
    scorer = BERTScorer(model_type="bert-base-uncased")
    for i in range(len(nurse_notes)):
        word = ""
        # BERTScore penalizes punctuation, we should take this into account - https://aclanthology.org/2023.findings-acl.381.pdf
        if i < 5:
            candidate =  [re.sub(r'[^\w\s/]', '', i) for i in met_dfs[f'{i % (len(nurse_notes) // 2)}_met']['report']]
            word = "Met"
        else:
            candidate =  [re.sub(r'[^\w\s/]', '', i) for i in unmet_dfs[f'{i % (len(nurse_notes) // 2)}_unmet']['report']]
            word = "Unmet"
        reference = [re.sub(r'[^\w\s/]', '', nurse_notes['Note'][i])] * (len(candidate))
        P, R, F1 = scorer.score(candidate, reference)
        # save the F1 values, as these are recommended for use by the BERTScore authors - http://arxiv.org/abs/1904.09675
        score_dictionary[f'{i}_{word.lower()}'] = F1.mean()
        print(f"{word} {i} F1 Score: {F1.mean()}")

    # put scores in a pickle file
    with open(savingFile, 'wb') as f:
        pickle.dump(score_dictionary, f)
    

In [5]:
bertScore(gpt4_load, gpt4_save_bert)

Met 0 F1 Score: 0.5195302963256836
Met 1 F1 Score: 0.4819844365119934
Met 2 F1 Score: 0.5033162832260132
Met 3 F1 Score: 0.5352760553359985
Met 4 F1 Score: 0.48810189962387085
Unmet 5 F1 Score: 0.45387497544288635
Unmet 6 F1 Score: 0.5249842405319214
Unmet 7 F1 Score: 0.5040232539176941
Unmet 8 F1 Score: 0.4989800453186035
Unmet 9 F1 Score: 0.48401233553886414


In [6]:
# load scores from pickle file - gpt4
with open(gpt4_save_bert, 'rb') as f:
    loaded_scores = pickle.load(f)
print(loaded_scores)

loaded_scores = pd.DataFrame(loaded_scores, index=['values'])
loaded_scores = loaded_scores.transpose()
print(f"Average BERTScore Met Notes GPT4: {float((((loaded_scores.iloc[:5])['values']).sum()) / (len((loaded_scores.iloc[:5])['values'])))}")
print(f"Average BERTScore Unmet Notes GPT4: {float((((loaded_scores.iloc[5:])['values']).sum()) / (len((loaded_scores.iloc[:5])['values'])))}")
print(f"Average BERTScore All Notes GPT4: {float((((loaded_scores)['values']).sum()) / (len((loaded_scores)['values'])))}")



{'0_met': tensor(0.5195), '1_met': tensor(0.4820), '2_met': tensor(0.5033), '3_met': tensor(0.5353), '4_met': tensor(0.4881), '5_unmet': tensor(0.4539), '6_unmet': tensor(0.5250), '7_unmet': tensor(0.5040), '8_unmet': tensor(0.4990), '9_unmet': tensor(0.4840)}
Average BERTScore Met Notes GPT4: 0.5056418180465698
Average BERTScore Unmet Notes GPT4: 0.4931749701499939
Average BERTScore All Notes GPT4: 0.4994083344936371


In [7]:
bertScore(gpt3_load, gpt3_save_bert)

Met 0 F1 Score: 0.5130299925804138
Met 1 F1 Score: 0.47652778029441833
Met 2 F1 Score: 0.4891226887702942
Met 3 F1 Score: 0.5416467785835266
Met 4 F1 Score: 0.5591109395027161
Unmet 5 F1 Score: 0.49972108006477356
Unmet 6 F1 Score: 0.5118075609207153
Unmet 7 F1 Score: 0.4898986220359802
Unmet 8 F1 Score: 0.472438246011734
Unmet 9 F1 Score: 0.47338348627090454


In [8]:
# load scores from pickle file - gpt3
with open(gpt3_save_bert, 'rb') as f:
    loaded_scores = pickle.load(f)
print(loaded_scores)
loaded_scores = pd.DataFrame(loaded_scores, index=['values'])
loaded_scores = loaded_scores.transpose()

print(f"Average BERTScore Met Notes GPT3: {float((((loaded_scores.iloc[:5])['values']).sum()) / (len((loaded_scores.iloc[:5])['values'])))}")
print(f"Average BERTScore Unmet Notes GPT3: {float((((loaded_scores.iloc[5:])['values']).sum()) / (len((loaded_scores.iloc[:5])['values'])))}")
print(f"Average BERTScore All Notes GPT3: {float((((loaded_scores)['values']).sum()) / (len((loaded_scores)['values'])))}")


{'0_met': tensor(0.5130), '1_met': tensor(0.4765), '2_met': tensor(0.4891), '3_met': tensor(0.5416), '4_met': tensor(0.5591), '5_unmet': tensor(0.4997), '6_unmet': tensor(0.5118), '7_unmet': tensor(0.4899), '8_unmet': tensor(0.4724), '9_unmet': tensor(0.4734)}
Average BERTScore Met Notes GPT3: 0.5158876180648804
Average BERTScore Unmet Notes GPT3: 0.48944979906082153
Average BERTScore All Notes GPT3: 0.5026687383651733


# Lexical Overlap Metric - METEOR

In [9]:
def calculate_meteor(loadingFile, savingFile):
    # load all files from data folder (generated in Google Colab)
    met_dfs = {}
    unmet_dfs = {}
    for file in os.listdir(loadingFile):
        if '.csv' in file:
            
            if 'unmet' in file:
                unmet_dfs[file.replace('.csv', '')] = pd.read_csv(f'{loadingFile}{file}')
            else:
                met_dfs[file.replace('.csv', '')] = pd.read_csv(f'{loadingFile}{file}')
    nurse_notes = pd.read_excel('./fake_notes.xlsx')

    score_dictionary = {}
    for i in range(len(nurse_notes)):
        meteor_scores = []
        word = ""
        if i < 5:
            candidates =  [re.sub(r'[^\w\s/]', '', i) for i in met_dfs[f'{i % (len(nurse_notes) // 2)}_met']['report']]
            word = "Met"
        else:
            candidates =  [re.sub(r'[^\w\s/]', '', i) for i in unmet_dfs[f'{i % (len(nurse_notes) // 2)}_unmet']['report']]
            word = "Unmet"
        reference = re.sub(r'[^\w\s/]', '', nurse_notes['Note'][i])
        for candidate in candidates:
            output_score = meteor([word_tokenize(candidate)], word_tokenize(reference))
            meteor_scores.append(output_score)
        score_dictionary[f'{i % (len(nurse_notes) // 2)}_{word}'] = (sum(meteor_scores)) / (len(meteor_scores))
    print(score_dictionary)

    # put scores in a pickle file
    with open(savingFile, 'wb') as f:
        pickle.dump(score_dictionary, f)

In [10]:
calculate_meteor(gpt3_load, gpt3_save_meteor)

{'0_Met': 0.18507853638767024, '1_Met': 0.13876725754795668, '2_Met': 0.12038962251667999, '3_Met': 0.2024498453413253, '4_Met': 0.2727674710529068, '0_Unmet': 0.09964158904819956, '1_Unmet': 0.1370909943208268, '2_Unmet': 0.11780806527150453, '3_Unmet': 0.09560695804278553, '4_Unmet': 0.14854885482654875}


In [11]:
# load scores from pickle file - gpt3
with open(gpt3_save_meteor, 'rb') as f:
    loaded_scores = pickle.load(f)
print(loaded_scores)
loaded_scores = pd.DataFrame(loaded_scores, index=['values'])
loaded_scores = loaded_scores.transpose()

print(f"Average METEOR Score Met Notes GPT3: {float((((loaded_scores.iloc[:5])['values']).sum()) / (len((loaded_scores.iloc[:5])['values'])))}")
print(f"Average METEOR Score Unmet Notes GPT3: {float((((loaded_scores.iloc[5:])['values']).sum()) / (len((loaded_scores.iloc[:5])['values'])))}")
print(f"Average METEOR Score All Notes GPT3: {float((((loaded_scores)['values']).sum()) / (len((loaded_scores)['values'])))}")

{'0_Met': 0.18507853638767024, '1_Met': 0.13876725754795668, '2_Met': 0.12038962251667999, '3_Met': 0.2024498453413253, '4_Met': 0.2727674710529068, '0_Unmet': 0.09964158904819956, '1_Unmet': 0.1370909943208268, '2_Unmet': 0.11780806527150453, '3_Unmet': 0.09560695804278553, '4_Unmet': 0.14854885482654875}
Average METEOR Score Met Notes GPT3: 0.1838905465693078
Average METEOR Score Unmet Notes GPT3: 0.11973929230197304
Average METEOR Score All Notes GPT3: 0.15181491943564043


In [12]:
calculate_meteor(gpt4_load, gpt4_save_meteor)

{'0_Met': 0.17525515679133688, '1_Met': 0.12145730054313354, '2_Met': 0.12057100196092599, '3_Met': 0.19037535582746445, '4_Met': 0.13427610942253193, '0_Unmet': 0.07119260017738657, '1_Unmet': 0.13787941160722675, '2_Unmet': 0.11924317257791932, '3_Unmet': 0.1296092848559098, '4_Unmet': 0.1524257676376596}


In [13]:
# load scores from pickle file - gpt4
with open(gpt4_save_meteor, 'rb') as f:
    loaded_scores = pickle.load(f)
print(loaded_scores)
loaded_scores = pd.DataFrame(loaded_scores, index=['values'])
loaded_scores = loaded_scores.transpose()

print(f"Average METEOR Score Met Notes GPT4: {float((((loaded_scores.iloc[:5])['values']).sum()) / (len((loaded_scores.iloc[:5])['values'])))}")
print(f"Average METEOR Score Unmet Notes GPT4: {float((((loaded_scores.iloc[5:])['values']).sum()) / (len((loaded_scores.iloc[:5])['values'])))}")
print(f"Average METEOR Score All Notes GPT4: {float((((loaded_scores)['values']).sum()) / (len((loaded_scores)['values'])))}")

{'0_Met': 0.17525515679133688, '1_Met': 0.12145730054313354, '2_Met': 0.12057100196092599, '3_Met': 0.19037535582746445, '4_Met': 0.13427610942253193, '0_Unmet': 0.07119260017738657, '1_Unmet': 0.13787941160722675, '2_Unmet': 0.11924317257791932, '3_Unmet': 0.1296092848559098, '4_Unmet': 0.1524257676376596}
Average METEOR Score Met Notes GPT4: 0.14838698490907856
Average METEOR Score Unmet Notes GPT4: 0.1220700473712204
Average METEOR Score All Notes GPT4: 0.1352285161401495


# Sentiment Analysis - TextBlob

In [14]:
def preprocess_text(text):
    text = re.sub(r'[^\w\s/]', '', text)
    # tokenize
    tokens = word_tokenize(text.lower())
    # remove stop words
    filtered_tokens = [token for token in tokens if token not in stopwords.words('english')]
    # lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    # join the tokens back into a string
    processed_text = ' '.join(lemmatized_tokens)
    return processed_text

In [40]:
def sentiment_interpreter(sentiment):
    sentiment = round(sentiment, 2)
    if sentiment > 0.5:
        return "positive"
    elif sentiment < -0.5: 
        return "negative"
    else:
        return "neutral"

In [41]:
def calculate_sentiment(loadingFile, savingFile):
    # load all files from data folder (generated in Google Colab)
    met_dfs = {}
    unmet_dfs = {}
    for file in os.listdir(loadingFile):
        if '.csv' in file:
            
            if 'unmet' in file:
                unmet_dfs[file.replace('.csv', '')] = pd.read_csv(f'{loadingFile}{file}')
            else:
                met_dfs[file.replace('.csv', '')] = pd.read_csv(f'{loadingFile}{file}')
    nurse_notes = pd.read_excel('./fake_notes.xlsx')

    score_dictionary = {}
    for i in range(len(nurse_notes)):
        nurse_note = preprocess_text(nurse_notes['Note'][i])
        nurse_blob = TextBlob(nurse_note)
        nurse_sentiment = nurse_blob.sentences[0].sentiment.polarity
        nurse_subjectivity = nurse_blob.sentences[0].sentiment.subjectivity
        word = ""
        if i < 5:
            candidates = [(TextBlob(preprocess_text(i))).sentences[0].sentiment.polarity for i in met_dfs[f'{i % (len(nurse_notes) // 2)}_met']['report']]
            generated_sentiment = (sum(candidates))/ len(candidates)
            candidates = [(TextBlob(preprocess_text(i))).sentences[0].sentiment.subjectivity for i in met_dfs[f'{i % (len(nurse_notes) // 2)}_met']['report']]
            generated_subjectivity = (sum(candidates))/ len(candidates)
            word = "Met"
        else:
            candidates = [(TextBlob(preprocess_text(i))).sentences[0].sentiment.polarity for i in unmet_dfs[f'{i % (len(nurse_notes) // 2)}_unmet']['report']]
            generated_sentiment = (sum(candidates))/ len(candidates)
            candidates = [(TextBlob(preprocess_text(i))).sentences[0].sentiment.subjectivity for i in unmet_dfs[f'{i % (len(nurse_notes) // 2)}_unmet']['report']]
            generated_subjectivity = (sum(candidates))/ len(candidates)
            word = "Unmet"
        score_dictionary[f'{i % (len(nurse_notes) // 2)}_{word}'] = {"Nurse Sentiment": nurse_sentiment, "Generated Sentiment": generated_sentiment, "Absolute Difference in Sentiment": math.sqrt((nurse_sentiment-generated_sentiment) ** 2), "Nurse Subjectivity": nurse_subjectivity , "Generated Subjectivity": generated_subjectivity, "Absolute Difference in Subjectivity":math.sqrt((nurse_subjectivity-generated_subjectivity) ** 2)}
    print(score_dictionary)
    # put scores in a pickle file
    with open(savingFile, 'wb') as f:
        pickle.dump(score_dictionary, f)

In [42]:
calculate_sentiment(gpt3_load, gpt3_save_sentiment)

{'0_Met': {'Nurse Sentiment': 0.023333333333333317, 'Generated Sentiment': 0.16859792459224276, 'Absolute Difference in Sentiment': 0.14526459125890945, 'Nurse Subjectivity': 0.4866666666666667, 'Generated Subjectivity': 0.4358268555238252, 'Absolute Difference in Subjectivity': 0.05083981114284147}, '1_Met': {'Nurse Sentiment': 0.43333333333333335, 'Generated Sentiment': 0.21274829857011676, 'Absolute Difference in Sentiment': 0.2205850347632166, 'Nurse Subjectivity': 0.5416666666666666, 'Generated Subjectivity': 0.3727011756198713, 'Absolute Difference in Subjectivity': 0.1689654910467953}, '2_Met': {'Nurse Sentiment': 0.24444444444444444, 'Generated Sentiment': 0.21828533059468935, 'Absolute Difference in Sentiment': 0.026159113849755083, 'Nurse Subjectivity': 0.2638888888888889, 'Generated Subjectivity': 0.4248683612266801, 'Absolute Difference in Subjectivity': 0.1609794723377912}, '3_Met': {'Nurse Sentiment': 0.08484848484848484, 'Generated Sentiment': 0.2149437280177979, 'Absolu

In [47]:
def sentiment_parser(loaded_scores):
    nurse_met_sentiment = 0
    generated_met_sentiment = 0
    nurse_met_subjectivity = 0
    generated_met_subjectivity = 0
    met_count = 0
    nurse_unmet_sentiment = 0
    generated_unmet_sentiment = 0
    nurse_unmet_subjectivity = 0
    generated_unmet_subjectivity = 0
    unmet_count = 0
    for score in loaded_scores:
        if 'Unmet' in score:
            nurse_unmet_sentiment += loaded_scores[score]['Nurse Sentiment']
            generated_unmet_sentiment += loaded_scores[score]['Generated Sentiment']
            nurse_unmet_subjectivity += loaded_scores[score]['Nurse Subjectivity']
            generated_unmet_subjectivity += loaded_scores[score]['Generated Subjectivity']
            unmet_count += 1

        elif 'Met' in score:
            # print(loaded_scores[score]['Generated Sentiment'])
            nurse_met_sentiment += loaded_scores[score]['Nurse Sentiment']
            generated_met_sentiment += loaded_scores[score]['Generated Sentiment']
            nurse_met_subjectivity += loaded_scores[score]['Nurse Subjectivity']
            generated_met_subjectivity += loaded_scores[score]['Generated Subjectivity']
            met_count += 1
    # met note scores
    print("\n------------\nMet Notes\n------------\n")
    print(f"Nurse Met Sentiment: {nurse_met_sentiment / met_count}")
    print(f"Generated Met Sentiment: {generated_met_sentiment / met_count}")
    print(f"Difference in Met Sentiment: {math.sqrt(((nurse_met_sentiment / met_count)-(generated_met_sentiment / met_count)) ** 2)}")
    print(f"Nurse Met Subjectivity: {nurse_met_subjectivity / met_count}")
    print(f"Generated Met Subjectivity: {generated_met_subjectivity / met_count}")
    print(f"Difference in Met Subjectivity: {math.sqrt(((nurse_met_subjectivity / met_count)-(generated_met_subjectivity / met_count)) ** 2)}")

    # unmet note scores
    print("\n------------\nUnmet Notes\n------------\n")
    print(f"Nurse Unmet Sentiment: {nurse_unmet_sentiment / unmet_count}")
    print(f"Generated Unmet Sentiment: {generated_unmet_sentiment / unmet_count}")
    print(f"Difference in Unmet Sentiment: {math.sqrt(((nurse_unmet_sentiment / unmet_count)-(generated_unmet_sentiment / unmet_count)) ** 2)}")
    print(f"Nurse Unmet Subjectivity: {nurse_unmet_subjectivity / unmet_count}")
    print(f"Generated Unmet Subjectivity: {generated_unmet_subjectivity / unmet_count}")
    print(f"Difference in Unmet Subjectivity: {math.sqrt(((nurse_unmet_subjectivity / unmet_count)-(generated_unmet_subjectivity / unmet_count)) ** 2)}")


In [48]:
# load scores from pickle file - gpt4
with open(gpt3_save_sentiment, 'rb') as f:
    loaded_scores = pickle.load(f)
print("GPT3")
print(loaded_scores)
sentiment_parser(loaded_scores=loaded_scores)

GPT3
{'0_Met': {'Nurse Sentiment': 0.023333333333333317, 'Generated Sentiment': 0.16859792459224276, 'Absolute Difference in Sentiment': 0.14526459125890945, 'Nurse Subjectivity': 0.4866666666666667, 'Generated Subjectivity': 0.4358268555238252, 'Absolute Difference in Subjectivity': 0.05083981114284147}, '1_Met': {'Nurse Sentiment': 0.43333333333333335, 'Generated Sentiment': 0.21274829857011676, 'Absolute Difference in Sentiment': 0.2205850347632166, 'Nurse Subjectivity': 0.5416666666666666, 'Generated Subjectivity': 0.3727011756198713, 'Absolute Difference in Subjectivity': 0.1689654910467953}, '2_Met': {'Nurse Sentiment': 0.24444444444444444, 'Generated Sentiment': 0.21828533059468935, 'Absolute Difference in Sentiment': 0.026159113849755083, 'Nurse Subjectivity': 0.2638888888888889, 'Generated Subjectivity': 0.4248683612266801, 'Absolute Difference in Subjectivity': 0.1609794723377912}, '3_Met': {'Nurse Sentiment': 0.08484848484848484, 'Generated Sentiment': 0.2149437280177979, 'A

In [45]:
calculate_sentiment(gpt4_load, gpt4_save_sentiment)

{'0_Met': {'Nurse Sentiment': 0.023333333333333317, 'Generated Sentiment': 0.1003668924310701, 'Absolute Difference in Sentiment': 0.07703355909773678, 'Nurse Subjectivity': 0.4866666666666667, 'Generated Subjectivity': 0.41893389166313416, 'Absolute Difference in Subjectivity': 0.06773277500353253}, '1_Met': {'Nurse Sentiment': 0.43333333333333335, 'Generated Sentiment': 0.12864769770302556, 'Absolute Difference in Sentiment': 0.3046856356303078, 'Nurse Subjectivity': 0.5416666666666666, 'Generated Subjectivity': 0.3797713193000078, 'Absolute Difference in Subjectivity': 0.16189534736665884}, '2_Met': {'Nurse Sentiment': 0.24444444444444444, 'Generated Sentiment': 0.1332921896915225, 'Absolute Difference in Sentiment': 0.11115225475292193, 'Nurse Subjectivity': 0.2638888888888889, 'Generated Subjectivity': 0.39848440505073157, 'Absolute Difference in Subjectivity': 0.13459551616184268}, '3_Met': {'Nurse Sentiment': 0.08484848484848484, 'Generated Sentiment': 0.09827200901324891, 'Abso

In [49]:
# load scores from pickle file - gpt4
with open(gpt4_save_sentiment, 'rb') as f:
    loaded_scores = pickle.load(f)
print("GPT4")
print(loaded_scores)
sentiment_parser(loaded_scores=loaded_scores)

GPT4
{'0_Met': {'Nurse Sentiment': 0.023333333333333317, 'Generated Sentiment': 0.1003668924310701, 'Absolute Difference in Sentiment': 0.07703355909773678, 'Nurse Subjectivity': 0.4866666666666667, 'Generated Subjectivity': 0.41893389166313416, 'Absolute Difference in Subjectivity': 0.06773277500353253}, '1_Met': {'Nurse Sentiment': 0.43333333333333335, 'Generated Sentiment': 0.12864769770302556, 'Absolute Difference in Sentiment': 0.3046856356303078, 'Nurse Subjectivity': 0.5416666666666666, 'Generated Subjectivity': 0.3797713193000078, 'Absolute Difference in Subjectivity': 0.16189534736665884}, '2_Met': {'Nurse Sentiment': 0.24444444444444444, 'Generated Sentiment': 0.1332921896915225, 'Absolute Difference in Sentiment': 0.11115225475292193, 'Nurse Subjectivity': 0.2638888888888889, 'Generated Subjectivity': 0.39848440505073157, 'Absolute Difference in Subjectivity': 0.13459551616184268}, '3_Met': {'Nurse Sentiment': 0.08484848484848484, 'Generated Sentiment': 0.09827200901324891, 